In [1]:
from pycaret.classification import *
from selenium import webdriver
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
import random
from datetime import datetime, timedelta
import pickle
from fuzzywuzzy import fuzz, process
from fractions import Fraction
import time
import collections
import concurrent.futures
from collections import defaultdict
import json
import requests
from bs4 import BeautifulSoup
import statistics
import math

C:\Users\idotz\anaconda3\envs\p37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\idotz\anaconda3\envs\p37\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
C:\Users\idotz\anaconda3\envs\p37\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)
C:\Users\idotz\anaconda3\envs\p37\lib\site-packages\llvmlite\llvmpy\__init__.py:4: UserWarning: The module `llvmlite.llvmpy` is deprecated and will be removed in the future.
  "The module `llvmlite.llvmpy` is deprecated and will be removed in the "
C:\Users\idotz\anaconda3\envs\p37\lib\site-packages\llvmlite\llvmpy\core.py:9: UserWarning: The module `llvmlite.llvmpy.core` is deprecated and will be removed in the future. Equivalent functionality is provided by `llvmlite.ir`.
  "The module `llvmlite.llvmpy.core` is deprecated and will be removed in "
C:\Users\idotz\anaconda3\envs\p37\

In [2]:
open_file = open('model_columns.pkl', "rb")
model_columns = list(pickle.load(open_file))
open_file.close()

In [3]:
scraped_cols = ['Total shots','Expected goals (xG)','Offsides','Accurate passes percentage','score','Corners','Fouls committed','Shots on target','Blocks']
pred_cols = dict(zip(model_columns, scraped_cols))

In [4]:
pred_cols

{'Shot Attempts': 'Total shots',
 'Total xG': 'Expected goals (xG)',
 'Offsides': 'Offsides',
 'Pass Completion': 'Accurate passes percentage',
 'Goals': 'score',
 'Corners': 'Corners',
 'Fouls Committed': 'Fouls committed',
 'Total Shots on Target': 'Shots on target',
 'Shots Blocked': 'Blocks'}

In [4]:
model = load_model('lr_model')

Transformation Pipeline and Model Successfully Loaded


In [43]:
today = f"{datetime.now():%Y%m%d}"
print(today)
game_data = fotmob_game_data(f'https://www.fotmob.com/?date={today}')
all_odds = get_all_odds_links()
days_data = connect_links(game_data, all_odds)

20220911


In [44]:
days_data

{'2022-09-11 07:08:00': [['fc-utrecht-vs-vitesse',
   'NETHERLANDS - EREDIVISIE',
   'https://www.fotmob.com/match/3900391/matchfacts/fc-utrecht-vs-vitesse',
   'https://www.oddschecker.com/football/netherlands/eredivisie/fc-utrecht-v-vitesse/winner']],
 '2022-09-11 07:23:00': [['atalanta-vs-cremonese',
   'ITALY - SERIE A',
   'https://www.fotmob.com/match/3919442/matchfacts/atalanta-vs-cremonese',
   'https://www.oddschecker.com/football/italy/serie-a/atalanta-v-cremonese/winner']],
 '2022-09-11 07:53:00': [['strasbourg-vs-clermont-foot',
   'FRANCE - LIGUE 1',
   'https://www.fotmob.com/match/3904507/matchfacts/strasbourg-vs-clermont-foot',
   'https://www.oddschecker.com/football/france/ligue-1/strasbourg-v-clermont-foot/winner']],
 '2022-09-11 08:23:00': [['westerlo-vs-anderlecht',
   'BELGIUM - FIRST DIVISION A',
   'https://www.fotmob.com/match/3917008/matchfacts/westerlo-vs-anderlecht',
   'https://www.oddschecker.com/football/belgium/jupiler-pro-league/westerlo-v-anderlecht/wi

In [46]:
days_data = {'2022-09-11 07:08:00': [['fc-utrecht-vs-vitesse',
   'NETHERLANDS - EREDIVISIE',
   'https://www.fotmob.com/match/3900391/matchfacts/fc-utrecht-vs-vitesse',
   'https://www.oddschecker.com/football/netherlands/eredivisie/fc-utrecht-v-vitesse/winner']],
 '2022-09-11 07:23:00': [['atalanta-vs-cremonese',
   'ITALY - SERIE A',
   'https://www.fotmob.com/match/3919442/matchfacts/atalanta-vs-cremonese',
   'https://www.oddschecker.com/football/italy/serie-a/atalanta-v-cremonese/winner']],
 '2022-09-11 07:53:00': [['strasbourg-vs-clermont-foot',
   'FRANCE - LIGUE 1',
   'https://www.fotmob.com/match/3904507/matchfacts/strasbourg-vs-clermont-foot',
   'https://www.oddschecker.com/football/france/ligue-1/strasbourg-v-clermont-foot/winner']],
 '2022-09-11 08:23:00': [['westerlo-vs-anderlecht',
   'BELGIUM - FIRST DIVISION A',
   'https://www.fotmob.com/match/3917008/matchfacts/westerlo-vs-anderlecht',
   'https://www.oddschecker.com/football/belgium/jupiler-pro-league/westerlo-v-anderlecht/winner'],
  ['1.fc-kaiserslautern-vs-sv-darmstadt',
   'GERMANY - 2. BUNDESLIGA',
   'https://www.fotmob.com/match/3904057/matchfacts/1.fc-kaiserslautern-vs-sv-darmstadt',
   'https://www.oddschecker.com/football/germany/bundesliga-2/kaiserslautern-v-sv-darmstadt-98/winner'],
  ['1.fc-magdeburg-vs-greuther-f%C3%BCrth',
   'GERMANY - 2. BUNDESLIGA',
   'https://www.fotmob.com/match/3904056/matchfacts/1.fc-magdeburg-vs-greuther-f%C3%BCrth',
   'https://www.oddschecker.com/football/germany/bundesliga-2/fc-magdeburg-v-spvgg-greuther-furth/winner']],
 '2022-09-11 08:53:00': [['real-madrid-vs-mallorca',
   'SPAIN - LALIGA',
   'https://www.fotmob.com/match/3917977/matchfacts/real-madrid-vs-mallorca',
   'https://www.oddschecker.com/football/spain/la-liga-primera/real-madrid-v-mallorca/winner'],
  ['aalborg-bk-vs-lyngby',
   'DENMARK - SUPERLIGAEN',
   'https://www.fotmob.com/match/3887486/matchfacts/aalborg-bk-vs-lyngby',
   'https://www.oddschecker.com/football/denmark/superligaen/aab-v-lyngby/winner'],
  ['nordsj%C3%A6lland-vs-midtjylland',
   'DENMARK - SUPERLIGAEN',
   'https://www.fotmob.com/match/3887487/matchfacts/nordsj%C3%A6lland-vs-midtjylland',
   'https://www.oddschecker.com/football/denmark/superligaen/fc-nordsjaelland-v-midtjylland/winner']],
 '2022-09-11 09:08:00': [['servette-vs-fc-z%C3%BCrich',
   'SWITZERLAND - SUPER LEAGUE',
   'https://www.fotmob.com/match/3905413/matchfacts/servette-vs-fc-z%C3%BCrich',
   'https://www.oddschecker.com/football/switzerland/super-league/servette-v-fc-zurich/winner']],
 '2022-09-11 09:23:00': [['austria-lustenau-vs-lask',
   'AUSTRIA - BUNDESLIGA',
   'https://www.fotmob.com/match/3917280/matchfacts/austria-lustenau-vs-lask',
   'https://www.oddschecker.com/football/austria/bundesliga/austria-lustenau-v-lask-linz/winner'],
  ['hartberg-vs-austria-wien',
   'AUSTRIA - BUNDESLIGA',
   'https://www.fotmob.com/match/3917281/matchfacts/hartberg-vs-austria-wien',
   'https://www.oddschecker.com/football/austria/bundesliga/hartberg-v-fk-austria-vienna/winner'],
  ['cambuur-vs-fc-groningen',
   'NETHERLANDS - EREDIVISIE',
   'https://www.fotmob.com/match/3900392/matchfacts/cambuur-vs-fc-groningen',
   'https://www.oddschecker.com/football/netherlands/eredivisie/cambuur-leeuwarden-v-fc-groningen/winner'],
  ['psv-eindhoven-vs-rkc-waalwijk',
   'NETHERLANDS - EREDIVISIE',
   'https://www.fotmob.com/match/3900390/matchfacts/psv-eindhoven-vs-rkc-waalwijk',
   'https://www.oddschecker.com/football/netherlands/eredivisie/psv-v-rkc/winner']],
 '2022-09-11 09:52:00': [['bologna-vs-fiorentina',
   'ITALY - SERIE A',
   'https://www.fotmob.com/match/3919443/matchfacts/bologna-vs-fiorentina',
   'https://www.oddschecker.com/football/italy/serie-a/bologna-v-fiorentina/winner'],
  ['lecce-vs-monza',
   'ITALY - SERIE A',
   'https://www.fotmob.com/match/3919449/matchfacts/lecce-vs-monza',
   'https://www.oddschecker.com/football/italy/serie-a/lecce-v-monza/winner']],
 '2022-09-11 09:53:00': [['sassuolo-vs-udinese',
   'ITALY - SERIE A',
   'https://www.fotmob.com/match/3919453/matchfacts/sassuolo-vs-udinese',
   'https://www.oddschecker.com/football/italy/serie-a/sassuolo-v-udinese/winner'],
  ['ac-ajaccio-vs-nice',
   'FRANCE - LIGUE 1',
   'https://www.fotmob.com/match/3904500/matchfacts/ac-ajaccio-vs-nice',
   'https://www.oddschecker.com/football/france/ligue-1/ac-ajaccio-v-nice/winner'],
  ['angers-vs-montpellier',
   'FRANCE - LIGUE 1',
   'https://www.fotmob.com/match/3904502/matchfacts/angers-vs-montpellier',
   'https://www.oddschecker.com/football/france/ligue-1/angers-v-montpellier/winner'],
  ['lorient-vs-nantes',
   'FRANCE - LIGUE 1',
   'https://www.fotmob.com/match/3904503/matchfacts/lorient-vs-nantes',
   'https://www.oddschecker.com/football/france/ligue-1/lorient-v-nantes/winner']],
 '2022-09-11 09:54:00': [['toulouse-vs-reims',
   'FRANCE - LIGUE 1',
   'https://www.fotmob.com/match/3904509/matchfacts/toulouse-vs-reims',
   'https://www.oddschecker.com/football/france/ligue-1/toulouse-v-reims/winner'],
  ['gif-sundsvall-vs-elfsborg',
   'SWEDEN - ALLSVENSKAN',
   'https://www.fotmob.com/match/3805098/matchfacts/gif-sundsvall-vs-elfsborg',
   'https://www.oddschecker.com/football/sweden/allsvenskan/gif-sundsvall-v-elfsborg/winner'],
  ['hammarby-vs-djurg%C3%A5rden',
   'SWEDEN - ALLSVENSKAN',
   'https://www.fotmob.com/match/3805093/matchfacts/hammarby-vs-djurg%C3%A5rden',
   'https://www.oddschecker.com/football/sweden/allsvenskan/hammarby-v-djurgarden/winner']],
 '2022-09-11 10:23:00': [['1.-fc-k%C3%B6ln-vs-union-berlin',
   'GERMANY - BUNDESLIGA',
   'https://www.fotmob.com/match/3903592/matchfacts/1.-fc-k%C3%B6ln-vs-union-berlin',
   'https://www.oddschecker.com/football/germany/bundesliga/cologne-v-union-berlin/winner']],
 '2022-09-11 10:53:00': [['cercle-brugge-vs-royal-antwerp',
   'BELGIUM - FIRST DIVISION A',
   'https://www.fotmob.com/match/3917011/matchfacts/cercle-brugge-vs-royal-antwerp',
   'https://www.oddschecker.com/football/belgium/jupiler-pro-league/cercle-brugge-v-antwerp/winner'],
  ['br%C3%B8ndby-if-vs-randers-fc',
   'DENMARK - SUPERLIGAEN',
   'https://www.fotmob.com/match/3887485/matchfacts/br%C3%B8ndby-if-vs-randers-fc',
   'https://www.oddschecker.com/football/denmark/superligaen/brondby-v-randers-fc/winner'],
  ['sivasspor-vs-istanbulspor',
   'TÜRKIYE - SUPER LIG',
   'https://www.fotmob.com/match/3932442/matchfacts/sivasspor-vs-istanbulspor',
   'https://www.oddschecker.com/football/turkey/super-lig/sivasspor-v-istanbulspor/winner']],
 '2022-09-11 11:08:00': [['elche-vs-athletic-club',
   'SPAIN - LALIGA',
   'https://www.fotmob.com/match/3917975/matchfacts/elche-vs-athletic-club',
   'https://www.oddschecker.com/football/spain/la-liga-primera/elche-v-athletic-bilbao/winner']],
 '2022-09-11 11:23:00': [['pacos-de-ferreira-vs-casa-pia',
   'PORTUGAL - LIGA PORTUGAL',
   'https://www.fotmob.com/match/3937438/matchfacts/pacos-de-ferreira-vs-casa-pia',
   'https://www.oddschecker.com/football/portugal/primeira-liga/pacos-ferreira-v-casa-pia/winner'],
  ['basel-vs-grasshopper',
   'SWITZERLAND - SUPER LEAGUE',
   'https://www.fotmob.com/match/3905411/matchfacts/basel-vs-grasshopper',
   'https://www.oddschecker.com/football/switzerland/super-league/basel-v-grasshoppers/winner'],
  ['young-boys-vs-lugano',
   'SWITZERLAND - SUPER LEAGUE',
   'https://www.fotmob.com/match/3905410/matchfacts/young-boys-vs-lugano',
   'https://www.oddschecker.com/football/switzerland/super-league/young-boys-v-lugano/winner']],
 '2022-09-11 11:38:00': [['az-alkmaar-vs-fc-twente',
   'NETHERLANDS - EREDIVISIE',
   'https://www.fotmob.com/match/3900393/matchfacts/az-alkmaar-vs-fc-twente',
   'https://www.oddschecker.com/football/netherlands/eredivisie/az-v-fc-twente/winner']],
 '2022-09-11 11:53:00': [['sk-austria-klagenfurt-vs-sturm-graz',
   'AUSTRIA - BUNDESLIGA',
   'https://www.fotmob.com/match/3917277/matchfacts/sk-austria-klagenfurt-vs-sturm-graz',
   'https://www.oddschecker.com/football/austria/bundesliga/austria-klagenfurt-v-sk-sturm-graz/winner']],
 '2022-09-11 11:58:00': [['rennes-vs-auxerre',
   'FRANCE - LIGUE 1',
   'https://www.fotmob.com/match/3904508/matchfacts/rennes-vs-auxerre',
   'https://www.oddschecker.com/football/france/ligue-1/rennes-v-auxerre/winner']],
 '2022-09-11 12:23:00': [['sc-freiburg-vs-borussia-mgladbach',
   'GERMANY - BUNDESLIGA',
   'https://www.fotmob.com/match/3903591/matchfacts/sc-freiburg-vs-borussia-mgladbach',
   'https://www.oddschecker.com/football/germany/bundesliga/sc-freiburg-v-borussia-mgladbach/winner'],
  ['malm%C3%B6-ff-vs-ifk-norrk%C3%B6ping',
   'SWEDEN - ALLSVENSKAN',
   'https://www.fotmob.com/match/3805097/matchfacts/malm%C3%B6-ff-vs-ifk-norrk%C3%B6ping',
   'https://www.oddschecker.com/football/sweden/allsvenskan/kalmar-ff-v-bk-hacken/winner'],
  ['varbergs-bois-fc-vs-aik',
   'SWEDEN - ALLSVENSKAN',
   'https://www.fotmob.com/match/3805081/matchfacts/varbergs-bois-fc-vs-aik',
   'https://www.oddschecker.com/football/sweden/allsvenskan/varbergs-bois-fc-v-aik/winner']],
 '2022-09-11 12:53:00': [['lazio-vs-hellas-verona',
   'ITALY - SERIE A',
   'https://www.fotmob.com/match/3919448/matchfacts/lazio-vs-hellas-verona',
   'https://www.oddschecker.com/football/italy/serie-a/lazio-v-verona/winner'],
  ['silkeborg-vs-agf-aarhus',
   'DENMARK - SUPERLIGAEN',
   'https://www.fotmob.com/match/3887488/matchfacts/silkeborg-vs-agf-aarhus',
   'https://www.oddschecker.com/football/denmark/superligaen/silkeborg-if-v-agf-aarhus/winner'],
  ['aalesund-vs-sarpsborg-08',
   'NORWAY - ELITESERIEN',
   'https://www.fotmob.com/match/3803250/matchfacts/aalesund-vs-sarpsborg-08',
   'https://www.oddschecker.com/football/norway/eliteserien/aalesund-v-sarpsborg/winner'],
  ['hamarkameratene-vs-viking',
   'NORWAY - ELITESERIEN',
   'https://www.fotmob.com/match/3803251/matchfacts/hamarkameratene-vs-viking',
   'https://www.oddschecker.com/football/norway/eliteserien/hamkam-v-viking-fk/winner']],
 '2022-09-11 12:54:00': [['jerv-vs-molde',
   'NORWAY - ELITESERIEN',
   'https://www.fotmob.com/match/3803253/matchfacts/jerv-vs-molde',
   'https://www.oddschecker.com/football/norway/eliteserien/fk-jerv-v-molde/winner'],
  ['kristiansund-vs-v%C3%A5lerenga',
   'NORWAY - ELITESERIEN',
   'https://www.fotmob.com/match/3803254/matchfacts/kristiansund-vs-v%C3%A5lerenga',
   'https://www.oddschecker.com/football/norway/eliteserien/aalesund-v-sarpsborg/winner'],
  ['troms%C3%B8-vs-bod%c3%b8%2fglimt',
   'NORWAY - ELITESERIEN',
   'https://www.fotmob.com/match/3803257/matchfacts/troms%C3%B8-vs-bod%c3%b8%2fglimt',
   'https://www.oddschecker.com/football/norway/eliteserien/tromso-v-bodo-glimt/winner']],
 '2022-09-11 13:23:00': [['getafe-vs-real-sociedad',
   'SPAIN - LALIGA',
   'https://www.fotmob.com/match/3917970/matchfacts/getafe-vs-real-sociedad',
   'https://www.oddschecker.com/football/spain/la-liga-primera/getafe-v-real-sociedad/winner'],
  ['union-st.gilloise-vs-genk',
   'BELGIUM - FIRST DIVISION A',
   'https://www.fotmob.com/match/3917010/matchfacts/union-st.gilloise-vs-genk',
   'https://www.oddschecker.com/football/belgium/jupiler-pro-league/union-saint-gilloise-v-genk/winner']],
 '2022-09-11 13:53:00': [['arouca-vs-boavista',
   'PORTUGAL - LIGA PORTUGAL',
   'https://www.fotmob.com/match/3937428/matchfacts/arouca-vs-boavista',
   'https://www.oddschecker.com/football/portugal/primeira-liga/fc-arouca-v-boavista/winner'],
  ['maritimo-vs-gil-vicente',
   'PORTUGAL - LIGA PORTUGAL',
   'https://www.fotmob.com/match/3937433/matchfacts/maritimo-vs-gil-vicente',
   'https://www.oddschecker.com/football/portugal/primeira-liga/maritimo-v-gil-vicente/winner'],
  ['gaziantep-fk-vs-%C3%BCmraniyespor',
   'TÜRKIYE - SUPER LIG',
   'https://www.fotmob.com/match/3932443/matchfacts/gaziantep-fk-vs-%C3%BCmraniyespor',
   'https://www.oddschecker.com/football/turkey/super-lig/gaziantep-fk-v-umraniyespor/winner'],
  ['kas%C4%B1mpa%C5%9Fa-vs-galatasaray',
   'TÜRKIYE - SUPER LIG',
   'https://www.fotmob.com/match/3932445/matchfacts/kas%C4%B1mpa%C5%9Fa-vs-galatasaray',
   'https://www.oddschecker.com/football/turkey/super-lig/kasimpasa-v-galatasaray/winner'],
  ['kayserispor-vs-antalyaspor',
   'TÜRKIYE - SUPER LIG',
   'https://www.fotmob.com/match/3932447/matchfacts/kayserispor-vs-antalyaspor',
   'https://www.oddschecker.com/football/turkey/super-lig/kayserispor-v-antalyaspor/winner']],
 '2022-09-11 14:53:00': [['feyenoord-vs-sparta-rotterdam',
   'NETHERLANDS - EREDIVISIE',
   'https://www.fotmob.com/match/3900394/matchfacts/feyenoord-vs-sparta-rotterdam',
   'https://www.oddschecker.com/football/netherlands/eredivisie/feyenoord-v-sparta-rotterdam/winner'],
  ['fk-haugesund-vs-rosenborg',
   'NORWAY - ELITESERIEN',
   'https://www.fotmob.com/match/3803252/matchfacts/fk-haugesund-vs-rosenborg',
   'https://www.oddschecker.com/football/norway/eliteserien/haugesund-v-rosenborg/winner']],
 '2022-09-11 15:38:00': [['juventus-vs-salernitana',
   'ITALY - SERIE A',
   'https://www.fotmob.com/match/3919447/matchfacts/juventus-vs-salernitana',
   'https://www.oddschecker.com/football/italy/serie-a/juventus-v-salernitana/winner'],
  ['monaco-vs-lyon',
   'FRANCE - LIGUE 1',
   'https://www.fotmob.com/match/3904501/matchfacts/monaco-vs-lyon',
   'https://www.oddschecker.com/football/france/ligue-1/monaco-v-lyon/winner']],
 '2022-09-11 15:53:00': [['real-betis-vs-villarreal',
   'SPAIN - LALIGA',
   'https://www.fotmob.com/match/3917969/matchfacts/real-betis-vs-villarreal',
   'https://www.oddschecker.com/football/spain/la-liga-primera/real-betis-v-villarreal/winner'],
  ['gent-vs-zulte-waregem',
   'BELGIUM - FIRST DIVISION A',
   'https://www.fotmob.com/match/3916997/matchfacts/gent-vs-zulte-waregem',
   'https://www.oddschecker.com/football/belgium/jupiler-pro-league/gent-v-zulte-waregem/winner']],
 '2022-09-11 16:23:00': [['rio-ave-vs-braga',
   'PORTUGAL - LIGA PORTUGAL',
   'https://www.fotmob.com/match/3937429/matchfacts/rio-ave-vs-braga',
   'https://www.oddschecker.com/football/portugal/primeira-liga/rio-ave-v-braga/winner']],
 '2022-09-11 17:53:00': [['cruz-azul-vs-mazatlan-fc',
   'MEXICO - LIGA MX APERTURA',
   'https://www.fotmob.com/match/3888619/matchfacts/cruz-azul-vs-mazatlan-fc',
   'https://www.oddschecker.com/football/world/mexico/liga-mx/cruz-azul-v-mazatlan-fc/winner']],
 '2022-09-11 19:58:00': [['atlas-vs-atletico-de-san-luis',
   'MEXICO - LIGA MX APERTURA',
   'https://www.fotmob.com/match/3888633/matchfacts/atlas-vs-atletico-de-san-luis',
   'https://www.oddschecker.com/football/world/mexico/liga-mx/atlas-v-atletico-san-luis/winner']],
 '2022-09-11 21:58:00': [['pachuca-vs-tijuana',
   'MEXICO - LIGA MX APERTURA',
   'https://www.fotmob.com/match/3888634/matchfacts/pachuca-vs-tijuana',
   'https://www.oddschecker.com/football/world/mexico/liga-mx/cf-pachuca-v-club-tijuana/winner']]}

In [ ]:
c = days_data.copy()
todays_data = []

while True:
    for clock in c.copy().keys():
        if datetime.strptime(clock, "%Y-%m-%d %H:%M:%S") <= datetime.now():
            stats_urls = [x[2] for x in c[clock]]
            odds_urls = [x[3] for x in c[clock]]
            print(stats_urls, odds_urls)
            with concurrent.futures.ThreadPoolExecutor() as executor:
                r = executor.map(stats_odds, stats_urls, odds_urls)
            for result in r:
                todays_data.append(result)
            del c[clock]
    print(f'{len(c)} halftimes left -- {len(todays_data)}\n')
    if len(c) == 0:
        print('all done')
        break
    time.sleep(60)

open_file = open(f'{today}_data.pkl', "wb")
pickle.dump(todays_data, open_file) 
open_file.close()

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes left -- 0

29 halftimes lef

['Draw', 'Atalanta', 'Cremonese']
[2.8875, 1.7, 7.5]
atalanta-vs-cremonese
Predicted winner is 1

draw: predicted prob: 28.299999999999997% - oddschecker: 34.63% (2.888) ---- bet no value
home: predicted prob: 47.8% - oddschecker: 58.82% (1.7) ---- bet no value
away: predicted prob: 24.0% - oddschecker: 13.33% (7.5) ---- bet 0.06155


27 halftimes left -- 2

27 halftimes left -- 2

27 halftimes left -- 2

27 halftimes left -- 2

27 halftimes left -- 2

27 halftimes left -- 2

27 halftimes left -- 2

27 halftimes left -- 2

27 halftimes left -- 2

27 halftimes left -- 2

27 halftimes left -- 2

27 halftimes left -- 2

27 halftimes left -- 2

27 halftimes left -- 2

27 halftimes left -- 2

27 halftimes left -- 2

27 halftimes left -- 2

27 halftimes left -- 2

27 halftimes left -- 2

27 halftimes left -- 2

27 halftimes left -- 2

27 halftimes left -- 2

27 halftimes left -- 2

27 halftimes left -- 2

27 halftimes left -- 2

27 halftimes left -- 2

27 halftimes left -- 2

27 halftimes le

['Draw', 'PSV', 'RKC']
[3.8, 1.4, 10.0]
psv-eindhoven-vs-rkc-waalwijk
Predicted winner is 1

draw: predicted prob: 28.999999999999996% - oddschecker: 26.32% (3.8) ---- bet 0.0182
home: predicted prob: 67.0% - oddschecker: 71.43% (1.4) ---- bet no value
away: predicted prob: 4.0% - oddschecker: 10.0% (10.0) ---- bet no value


['Draw', 'Austria Lustenau', 'LASK Linz']
[6.0, 17.0, 1.1742424242424243]
austria-lustenau-vs-lask
Predicted winner is 2

draw: predicted prob: 19.7% - oddschecker: 16.67% (6.0) ---- bet 0.0182
home: predicted prob: 16.5% - oddschecker: 5.88% (17.0) ---- bet 0.0564
away: predicted prob: 63.800000000000004% - oddschecker: 85.18% (1.174) ---- bet no value


['Draw', 'Hartberg', 'FK Austria Vienna']
[5.25, 13.0, 1.2222222222222223]
hartberg-vs-austria-wien
Predicted winner is 2

draw: predicted prob: 8.0% - oddschecker: 19.05% (5.25) ---- bet no value
home: predicted prob: 1.9% - oddschecker: 7.69% (13.0) ---- bet no value
away: predicted prob: 90.10000000000001% - o

18 halftimes left -- 24

['https://www.fotmob.com/match/3917011/matchfacts/cercle-brugge-vs-royal-antwerp', 'https://www.fotmob.com/match/3887485/matchfacts/br%C3%B8ndby-if-vs-randers-fc', 'https://www.fotmob.com/match/3932442/matchfacts/sivasspor-vs-istanbulspor'] ['https://www.oddschecker.com/football/belgium/jupiler-pro-league/cercle-brugge-v-antwerp/winner', 'https://www.oddschecker.com/football/denmark/superligaen/brondby-v-randers-fc/winner', 'https://www.oddschecker.com/football/turkey/super-lig/sivasspor-v-istanbulspor/winner']
['Draw', 'Sivasspor', 'Istanbulspor']
[2.375, 2.2, 4.75]
sivasspor-vs-istanbulspor
Predicted winner is 0

draw: predicted prob: 34.8% - oddschecker: 42.11% (2.375) ---- bet no value
home: predicted prob: 34.699999999999996% - oddschecker: 45.45% (2.2) ---- bet no value
away: predicted prob: 30.5% - oddschecker: 21.05% (4.75) ---- bet 0.05985


['Draw', 'Cercle Brugge', 'Antwerp']
[3.75, 8.0, 1.4444444444444444]
cercle-brugge-vs-royal-antwerp
Predicted wi

['Draw', 'Kalmar FF', 'BK Hacken']
[3.4, 2.9, 2.3]
malm%C3%B6-ff-vs-ifk-norrk%C3%B6ping
Predicted winner is 0

draw: predicted prob: 40.0% - oddschecker: 29.41% (3.4) ---- bet 0.075
home: predicted prob: 28.999999999999996% - oddschecker: 34.48% (2.9) ---- bet no value
away: predicted prob: 31.1% - oddschecker: 43.48% (2.3) ---- bet no value


11 halftimes left -- 37

11 halftimes left -- 37

11 halftimes left -- 37

11 halftimes left -- 37

11 halftimes left -- 37

11 halftimes left -- 37

11 halftimes left -- 37

11 halftimes left -- 37

11 halftimes left -- 37

11 halftimes left -- 37

11 halftimes left -- 37

11 halftimes left -- 37

11 halftimes left -- 37

11 halftimes left -- 37

11 halftimes left -- 37

11 halftimes left -- 37

11 halftimes left -- 37

11 halftimes left -- 37

11 halftimes left -- 37

11 halftimes left -- 37

11 halftimes left -- 37

11 halftimes left -- 37

11 halftimes left -- 37

11 halftimes left -- 37

11 halftimes left -- 37

11 halftimes left -- 37

11 h

['Draw', 'Gaziantep FK', 'Umraniyespor']
[2.4, 2.75, 3.225]
gaziantep-fk-vs-%C3%BCmraniyespor
Predicted winner is 2

draw: predicted prob: 24.6% - oddschecker: 41.67% (2.4) ---- bet no value
home: predicted prob: 26.5% - oddschecker: 36.36% (2.75) ---- bet no value
away: predicted prob: 48.9% - oddschecker: 31.01% (3.225) ---- bet 0.12965


['Draw', 'Kasimpasa', 'Galatasaray']
[3.0, 5.5, 1.75]
kas%C4%B1mpa%C5%9Fa-vs-galatasaray
Predicted winner is 2

draw: predicted prob: 17.9% - oddschecker: 33.33% (3.0) ---- bet no value
home: predicted prob: 14.7% - oddschecker: 18.18% (5.5) ---- bet no value
away: predicted prob: 67.4% - oddschecker: 57.14% (1.75) ---- bet 0.11965


['Draw', 'Kayserispor', 'Antalyaspor']
[2.3, 3.3, 2.8]
kayserispor-vs-antalyaspor
Predicted winner is 1

draw: predicted prob: 27.700000000000003% - oddschecker: 43.48% (2.3) ---- bet no value
home: predicted prob: 38.3% - oddschecker: 30.3% (3.3) ---- bet 0.05735
away: predicted prob: 34.0% - oddschecker: 35.71% (2.8) 

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left -- 58

3 halftimes left

In [278]:
open_file = open('20220828_data.pkl', "rb")
xcxc = pickle.load(open_file)
open_file.close()

In [5]:
def predicted_probs(results):
    d = {}
    for k, v in results.items():
        try:
            d[k] = float(v[0]) - float(v[1])
        except:
            d[k] = v
    for_pred = [[d[key] for key in pred_cols.values()]]
    pred = model[-1].predict_proba(for_pred)
    return pred

In [6]:
def make_bet(decimal_odds, predicted_prob, k_weight):
    if 1/decimal_odds >= predicted_prob:
        return 'no value'
    else:
        return kelly(predicted_prob, decimal_odds, k_weight)

In [7]:
def print_advice(results):
    try:
        d_h_a = ['draw'] + results['odds_url'].split('/')[-2].split('-v-')
        order_decimal_odds = [getname(odds_name, list(results['odds'].keys())) for odds_name in d_h_a]
        print(order_decimal_odds)
        p_probs = predicted_probs(results)[0]
        predicted = np.argmax(p_probs)
        decimal_odds = [results['odds'][key] for key in order_decimal_odds]
        print(decimal_odds)
        bet_made = {}
        print('\033[1m' + results['match_name'] + '\033[0m')
        print(f'Predicted winner is {predicted}\n')
        for p_prob, d_odd, win in zip(p_probs, decimal_odds, ['draw', 'home','away']):
            p_prob = round(p_prob, 3)
            d_odd = round(d_odd, 3)
            size = make_bet(d_odd, p_prob, 0.5)
            print(f'{win}: predicted prob: {p_prob*100}% - oddschecker: {round(1/d_odd*100,2)}% ({d_odd}) ---- bet {size}')
            bet_made[win] = size
        print('\n')
        bet_made['odds_checker_sum'] = np.sum([1/o for o in decimal_odds])
        bet_made['winner'] = predicted
        bet_made['predicted_probs'] = np.round(p_probs, 3)
        bets = {'bet':bet_made} 
        row = {**results, **bets}
    except:
        row = results
    return row

In [8]:
def kelly(p, b, weight):
    stake = round((p * (b - 1) - (1 - p)) / (b - 1), 4) * weight
    return stake if stake > 0.0 else 'no value'

In [9]:
def getname(name, l):
    if len(l) == 0:
        return ''
    highest = process.extractOne(name,l)
    return highest[0]

In [10]:
def get_stats(url):
    try:
        page = requests.get(url)
        time.sleep(random.randint(0, 4))
        soup = BeautifulSoup(page.content, 'html.parser')
        ht = soup.find("span", {"class": "css-lv1jm0-bottomRow"}).text
        if ht != 'Half-Time':
                print('not halftime just yet ... sleeping\n')
                time.sleep(3*60)
                ht = soup.find("span", {"class": "css-lv1jm0-bottomRow"}).text
        div1 = soup.find_all("li", {"class": "css-radwzz-Stat e683amr6"})
        all_stats = {k[1]: [k[0], k[2]] for k in [z.split('|') for z in [x for x in [d.get_text('|') for d in div1]
                  if x not in ['Shots','Passes','Expected goals (xG)','Discipline','Defence','Duels']]]}
        score = soup.find_all("span", {"class": "css-bw7eig-topRow"})[0].text.split(' - ')
        score = [score[0], score[1]]
        posession = soup.find("div", {"class": "css-7s52se-PossessionWheel e683amr3"}).get_text('|').split('|')
        posession = [posession[1], posession[3]]
        all_stats['posession'] = posession
        all_stats['score'] = score
        game_stats = {}
        for name, stats in all_stats.items():
            if '(' in ''.join(stats):
                s = stats[0].split(' ') + stats[1].split(' ')
                if len(s) == 4:
                    s = [i.replace('(', '').replace(')','').replace('%','') for i in s]
                    game_stats[name] = [s[0], s[2]]
                    game_stats[name + ' percentage'] = [int(s[1])/100, int(s[3])/100]
                else:
                    s = [i for i in s if '%' not in i]
                    game_stats[name] = s
            else:
                game_stats[name] = stats
        match_name = url.split('/')[-1]
        game_stats['match_name'] = match_name
        game_stats['half_time'] = ht
        game_stats['url'] = url
    except Exception as e:
        print(e)
        game_stats = {}
    return game_stats

In [36]:
get_stats('https://www.fotmob.com/match/3900365/matchfacts/az-alkmaar-vs-nec-nijmegen')

not halftime just yet ... sleeping



{'Expected goals (xG)': ['2.62', '0.07'],
 'Total shots': ['30', '4'],
 'Big chances': ['4', '0'],
 'Big chances missed': ['4', '0'],
 'Accurate passes': ['534', '266'],
 'Accurate passes percentage': [0.89, 0.76],
 'Fouls committed': ['7', '10'],
 'Offsides': ['1', '0'],
 'Corners': ['8', '3'],
 'Shots off target': ['8', '2'],
 'Shots on target': ['13', '2'],
 'Blocked shots': ['9', '0'],
 'Hit woodwork': ['0', '0'],
 'Shots inside box': ['17', '1'],
 'Shots outside box': ['13', '3'],
 'xG first half': ['0.37', '0.04'],
 'xG second half': ['2.25', '0.03'],
 'xG open play': ['2.35', '0.04'],
 'xG set play': ['0.27', '0.03'],
 'xG on target (xGOT)': ['1.74', '0.89'],
 'Passes': ['600', '349'],
 'Own half': ['197', '199'],
 'Opposition half': ['337', '67'],
 'Accurate long balls': ['29', '23'],
 'Accurate long balls percentage': [0.59, 0.4],
 'Accurate crosses': ['6', '3'],
 'Accurate crosses percentage': [0.33, 0.33],
 'Throws': ['18', '13'],
 'Yellow cards': ['1', '2'],
 'Red cards': [

In [41]:
xg_fm = ['UNITED STATES - MAJOR LEAGUE SOCCER', 'ENGLAND - CHAMPIONSHIP',
 'ENGLAND - PREMIER LEAGUE', 'GERMANY - BUNDESLIGA', 'FRANCE - LIGUE 1', 'AUSTRIA - BUNDESLIGA', 'PORTUGAL - LIGA PORTUGAL',
 'BELGIUM - FIRST DIVISION A', 'SWITZERLAND - SUPER LEAGUE', 'SPAIN - LALIGA', 'DENMARK - SUPERLIGAEN', 'MEXICO - LIGA MX APERTURA',
 'NETHERLANDS - EREDIVISIE', 'SCOTLAND - PREMIERSHIP', 'ITALY - SERIE A', 'CHAMPIONS LEAGUE GRP. A', 'CHAMPIONS LEAGUE GRP. B', 'CHAMPIONS LEAGUE GRP. C',
 'CHAMPIONS LEAGUE GRP. D', 'CHAMPIONS LEAGUE GRP. E', 'CHAMPIONS LEAGUE GRP. F', 'CHAMPIONS LEAGUE GRP. G',
 'CHAMPIONS LEAGUE GRP. H', 'EUROPA LEAGUE GRP. A', 'EUROPA LEAGUE GRP. B', 'EUROPA LEAGUE GRP. C', 'EUROPA LEAGUE GRP. D',
 'EUROPA LEAGUE GRP. E', 'EUROPA LEAGUE GRP. F', 'EUROPA LEAGUE GRP. G', 'EUROPA LEAGUE GRP. H', 'EUROPA CONFERENCE LEAGUE GRP. A',
 'EUROPA CONFERENCE LEAGUE GRP. B', 'EUROPA CONFERENCE LEAGUE GRP. C', 'EUROPA CONFERENCE LEAGUE GRP. D', 'EUROPA CONFERENCE LEAGUE GRP. E',
         'EUROPA CONFERENCE LEAGUE GRP. F', 'EUROPA CONFERENCE LEAGUE GRP. G', 'EUROPA CONFERENCE LEAGUE GRP. H',
        'NORWAY - ELITESERIEN','TÜRKIYE - SUPER LIG', 'GERMANY - 2. BUNDESLIGA','SWEDEN - ALLSVENSKAN']

def fotmob_game_data(url):
    driver = webdriver.Chrome()
    driver.implicitly_wait(3)
    driver.get(url)
    divs = driver.find_elements(By.XPATH, '//div[@class="css-1f0alcw-Group e7pc1841"]')
    today = url[-8:]
    league = {}
    for count, bucket in enumerate(divs):
        league_name = bucket.find_elements(By.XPATH, '//a[@class="css-vfisw6-GroupTitleLink e7pc1842"]')[count].text
        if league_name not in xg_fm:
            continue
        url = bucket.find_elements(By.TAG_NAME, 'a')
        urls = [u.get_attribute('href') for u in url]
        time = bucket.text
        time = [datetime.strptime(i + today, '%H:%M%Y%m%d') + timedelta(minutes=53) for i in time.split('\n') if ':' in i]
        urls = urls[1:]
        names = [u.split('/')[-1] for u in urls]
        league[league_name] = {n: [u,t] for u, t, n in zip(urls, time, names)}
    return league

In [42]:
xg_leagues_o = {'UNITED STATES - MAJOR LEAGUE SOCCER':'world/usa/mls','ENGLAND - CHAMPIONSHIP':'english/championship',
              'ENGLAND - PREMIER LEAGUE': 'english/premier-league', 'GERMANY - BUNDESLIGA': 'germany/bundesliga',
              'FRANCE - LIGUE 1': 'france/ligue-1','AUSTRIA - BUNDESLIGA': 'austria/bundesliga',
              'PORTUGAL - LIGA PORTUGAL':'portugal/primeira-liga', 'BELGIUM - FIRST DIVISION A':'belgium/jupiler-pro-league',
              'SWITZERLAND - SUPER LEAGUE':'switzerland/super-league','SPAIN - LALIGA':'spain/la-liga-primera',
             'DENMARK - SUPERLIGAEN':'denmark/superligaen', 'MEXICO - LIGA MX APERTURA':'world/mexico/liga-mx',
             'NETHERLANDS - EREDIVISIE':'netherlands/eredivisie', 'SCOTLAND - PREMIERSHIP': 'scottish/premiership',
             'ITALY - SERIE A':'italy/serie-a','CHAMPIONS LEAGUE': 'champions-league','EUROPA LEAGUE': 'europa-league',
             'EUROPA CONFERENCE LEAGUE': 'uefa-europa-conference-league', 'NORWAY - ELITESERIEN':'norway/eliteserien',
               'TÜRKIYE - SUPER LIG':'turkey/super-lig','GERMANY - 2. BUNDESLIGA':'germany/bundesliga-2',
               'SWEDEN - ALLSVENSKAN': 'sweden/allsvenskan'}

def get_all_odds_links():
    odds_links = {}
    for f_league, url_league in xg_leagues_o.items():
        driver = webdriver.Chrome()
        driver.implicitly_wait(3)
        url = 'https://www.oddschecker.com/football/' + url_league
        driver.get(url)
        driver.implicitly_wait(3)
        divs = driver.find_elements(By.XPATH, '//a[@class="beta-callout full-height-link whole-row-link"]')
        mt = []
        for div in divs:
            odds_url = div.get_attribute('href')
            mt.append(odds_url)
        odds_links[f_league] = mt
    return odds_links

In [13]:
def connect_links(data, all_odds):
    for l in data.keys():
        fotmob_names = data[l].keys()
        odds_names = {v: k for k, v in zip(all_odds[l],[str(i).split('/')[-2] for i in all_odds[l]])}
        for game in fotmob_names:
            match = getname(game, odds_names.keys())
            try:
                link = odds_names[match]
            except:
                link = ''
            data[l][game] = data[l][game] + [link]
    scraper_data = collections.defaultdict(list)
    for l_name, data in data.items():
        for game_name, l1_t_l2 in data.items():
            scraper_data[str(l1_t_l2[1])].append([game_name, l_name, l1_t_l2[0], l1_t_l2[2]])
    r = {}
    test = scraper_data.copy()
    for date, data in test.copy().items():
        if len(data) > 5:
            x = [data[i:i + 4] for i in range(0, len(data), 4)]
            d = datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
            groups = len(x)
            times = [str(d + timedelta(minutes = i * 1)) for i in range(-groups//2+1,groups//2+1, 1)]
            new = dict(zip(times, x))
            del test[date]
            r = {**r, **new}
        else:
            r[date] = data
    scraper_data = dict(sorted(r.items(), key=lambda x: x[0]))
    return scraper_data

In [14]:
def get_odds(url):
    try:
        driver = webdriver.Chrome()
        if url == '':
            return {'odds_url':url, 'odds': '', 'full_odds': ''}
        driver.implicitly_wait(1)
        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html, features="html.parser")
        odds_containers = soup.find_all("div", {"class": "oddsAreaWrapper_o17xb9rs RowLayout_refg9ta"})
        names = soup.find_all("div", {"class": "BetRowLeftBetContent_b2f00kt"})
        names = [n.text for n in names]
        odds_containers = [container.find_all('button') for container in odds_containers]
        odds = [[z.text for z in container] for container in odds_containers]
        odds = [[float(Fraction(j.replace(' ',''))) + 1.0 for j in o][:-3] for o in odds]
        median_odds = [statistics.median(o) for o in odds]
        median_odds = dict(zip(names, median_odds))
        odds = {'odds_url':url, 'odds': median_odds, 'full_odds': odds}
    except Exception as e:
        print(e)
        return {'odds_url':url, 'odds': '', 'full_odds': ''}
    driver.quit()
    return odds

In [50]:
get_odds('https://www.oddschecker.com/football/europa-conference-league/anderlecht-v-silkeborg-if/winner')

{'odds_url': 'https://www.oddschecker.com/football/europa-conference-league/anderlecht-v-silkeborg-if/winner',
 'odds': {'Anderlecht': 1.7272727272727273, 'Draw': 3.3, 'Silkeborg IF': 5.0},
 'full_odds': [[1.7272727272727273,
   1.7272727272727273,
   1.6666666666666665,
   1.65,
   1.7272727272727273,
   1.6666666666666665,
   1.6666666666666665,
   1.75,
   1.6666666666666665,
   1.75,
   1.7272727272727273,
   1.7857142857142856,
   1.7,
   1.7647058823529411,
   1.8,
   1.7,
   1.6666666666666665,
   1.6666666666666665,
   1.75,
   1.7,
   1.75,
   1.6666666666666665,
   1.8333333333333335],
  [3.4,
   3.25,
   3.4,
   3.4,
   3.0,
   3.4,
   3.3,
   3.3,
   3.5,
   3.3,
   3.25,
   3.5454545454545454,
   3.1,
   3.1538461538461537,
   3.2,
   3.4,
   3.5,
   3.1,
   3.5,
   3.3,
   3.25,
   3.3,
   3.6470588235294117],
  [5.0,
   5.0,
   5.5,
   4.8,
   4.5,
   5.5,
   4.8,
   5.0,
   5.0,
   4.75,
   5.0,
   5.7,
   4.75,
   4.75,
   4.846153846153847,
   5.0,
   5.0,
   4.6,
   

In [15]:
def stats_odds(stats_url, odds_url):
    stats = get_stats(stats_url)
    time.sleep(20)
    odds = get_odds(odds_url)
    row = {**stats, **odds}
    result = print_advice(row)
    return result